# Airbus Hackatuna

In [1]:
# =====================================================
# INITIALISATION ENVIRONNEMENT PROJET (PROPRE)
# =====================================================

import os
import sys
import warnings
import importlib
import csv
import numpy as np

from tqdm import tqdm

import h5py

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import os
import sys
sys.path.append("../")
sys.path.append("../src")

import config as c
import utils_N as u
import network_N as n
import lidar_utils

warnings.filterwarnings("ignore")

In [ ]:
# === Chemins absolus projet ===
#PROJECT_ROOT = r"C:\Users\Léopold\Downloads\Airbus-Hackathon-2026-main\Airbus-Hackathon-2026-main"
#SRC_PATH = os.path.join(PROJECT_ROOT, "src")

## === Nettoyage modules potentiellement mal chargés ===
#for module in ["config", "utils_N", "lidar_utils", "network"]:
#    if module in sys.modules:
#        del sys.modules[module]
#
## === Injection chemins prioritaires ===
#if PROJECT_ROOT not in sys.path:
#    sys.path.insert(0, PROJECT_ROOT)
#
#if SRC_PATH not in sys.path:
#    sys.path.insert(0, SRC_PATH)

# =====================================================
# IMPORTS PROJET
# =====================================================



# =====================================================
# VERIFICATION DES FICHIERS CHARGÉS
# =====================================================
#
#print("CONFIG FILE →", c.__file__)
#print("UTILS FILE  →", u.__file__)
#print("LIDAR FILE  →", lu.__file__)
#print("NETWORK FILE→", n.__file__)
#
#print("\nDataset class →", n.PointCloudDataset)
#print("Model class   →", n.PointNetSeg)


CONFIG FILE → C:\Users\Léopold\Downloads\Airbus-Hackathon-2026-main\Airbus-Hackathon-2026-main\config.py
UTILS FILE  → C:\Users\Léopold\Downloads\Airbus-Hackathon-2026-main\Airbus-Hackathon-2026-main\src\utils_N.py
LIDAR FILE  → C:\Users\Léopold\Downloads\Airbus-Hackathon-2026-main\Airbus-Hackathon-2026-main\src\lidar_utils.py
NETWORK FILE→ C:\Users\Léopold\Downloads\Airbus-Hackathon-2026-main\Airbus-Hackathon-2026-main\src\network.py

Dataset class → <class 'network.PointCloudDataset'>
Model class   → <class 'network.PointNetSeg'>


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = n.PointNetSeg(num_classes = 4).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
weights = torch.tensor([1.0, 8.0, 1.0, 1.0]).to(device)
criterion = nn.CrossEntropyLoss(weight=weights)

In [3]:
# Charger poids sauvegardés
model_path = "../models/PointNetSeg_cpu.pth"  # adapte si besoin

model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

print("✅ Modèle chargé et en mode évaluation")
test_ds = n.PointCloudTestDataset("../datasets/processed/test.h5")
test_loader = DataLoader(test_ds, batch_size = 1, shuffle=False)

print("✅ Test loader prêt")


✅ Modèle chargé et en mode évaluation
✅ Test loader prêt


In [6]:
def predict_segmentation(model, dataloader, device):

    model.eval()

    all_preds = []
    all_points = []
    all_poses = []

    with torch.no_grad():
        for points, labels, poses in dataloader:

            points = points.to(device)

            logits = model(points)
            preds = torch.argmax(logits, dim=2)

            all_preds.extend(preds.cpu().numpy())
            all_points.extend(points.cpu().numpy())
            all_poses.extend(poses.cpu().numpy())

    return all_preds, all_points, all_poses


In [ ]:
import open3d as o3d
import numpy as np

def cluster_by_class(frame_points, frame_preds, class_id, eps=1.2, min_points=20):

    mask = frame_preds == class_id
    class_points = frame_points[mask]

    if len(class_points) < min_points:
        return []

    class_points = class_points[:, :3].astype(np.float64)

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(class_points)

    labels = np.array(
        pcd.cluster_dbscan(eps = eps, min_points = min_points)
    )

    clusters = []

    for k in np.unique(labels):
        if k == -1:
            continue
        clusters.append(class_points[labels == k])

    return clusters


In [14]:
def compute_obb(cluster_pts):

    if cluster_pts.shape[0] < 5:
        return None

    cluster_pts = cluster_pts[:, :3].astype(np.float64)

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cluster_pts)

    try:
        obb = pcd.get_oriented_bounding_box()

        center = obb.center
        extent = obb.extent
        R = obb.R

        yaw = np.arctan2(R[1,0], R[0,0])

        return {
            "center": center,
            "extent": extent,
            "yaw": yaw
        }

    except:
        return None


In [19]:
def full_inference_pipeline(model, dataloader, device):

    preds, points, poses = predict_segmentation(model, dataloader, device)

    detections = []

    for i in range(len(poses)):

        frame_points = points[i]
        frame_preds  = preds[i]
        frame_pose   = poses[i]

        for class_id in [0,1,2,3]:

            clusters = cluster_by_class(
                frame_points,
                frame_preds,
                class_id
            )

            for cluster_pts in clusters:

                obb = compute_obb(cluster_pts)

                if obb is None:
                    continue

                detections.append({
                    "ego": frame_pose,
                    "center": obb["center"],
                    "extent": obb["extent"],
                    "yaw": obb["yaw"],
                    "class_id": class_id
                })

    return detections


In [22]:
preds, points, poses = predict_segmentation(model, test_loader, device)

In [25]:
len(points)

178

In [20]:
detections = full_inference_pipeline(model, test_loader, device)

print("Nombre total de bounding boxes détectées :", len(detections))

Nombre total de bounding boxes détectées : 3024


In [16]:
def full_gt_pipeline(dataloader):

    detections_gt = []

    for points, labels, poses in dataloader:

        points = points.numpy()
        labels = labels.numpy()
        poses  = poses.numpy()

        for i in range(len(points)):

            frame_points = points[i]
            frame_labels = labels[i]
            frame_pose   = poses[i]

            for class_id in [0,1,2,3]:

                clusters = cluster_by_class(
                    frame_points,
                    frame_labels,
                    class_id
                )

                for cluster_pts in clusters:

                    obb = compute_obb(cluster_pts)

                    if obb is None:
                        continue

                    detections_gt.append({
                        "ego": frame_pose,
                        "center": obb["center"],
                        "extent": obb["extent"],
                        "yaw": obb["yaw"],
                        "class_id": class_id
                    })

    return detections_gt


In [17]:
def compute_iou_3d(box1, box2):

    c1, e1 = box1["center"], box1["extent"]
    c2, e2 = box2["center"], box2["extent"]

    min1 = c1 - e1/2
    max1 = c1 + e1/2
    min2 = c2 - e2/2
    max2 = c2 + e2/2

    inter_min = np.maximum(min1, min2)
    inter_max = np.minimum(max1, max2)

    inter_dim = np.maximum(inter_max - inter_min, 0)
    inter_vol = np.prod(inter_dim)

    vol1 = np.prod(e1)
    vol2 = np.prod(e2)

    union = vol1 + vol2 - inter_vol

    if union == 0:
        return 0

    return inter_vol / union


In [18]:
def evaluate_object_iou(detections_pred, detections_gt, threshold=0.5):

    ious = []
    matches = 0

    for pred in detections_pred:

        best_iou = 0

        for gt in detections_gt:

            if not np.allclose(pred["ego"], gt["ego"]):
                continue

            if pred["class_id"] != gt["class_id"]:
                continue

            iou = compute_iou_3d(pred, gt)

            if iou > best_iou:
                best_iou = iou

        ious.append(best_iou)

        if best_iou > threshold:
            matches += 1

    mean_iou = np.mean(ious) if len(ious) > 0 else 0

    print("Mean Object IoU :", mean_iou)
    print("Correct detections (IoU>0.5):", matches)

    return mean_iou


In [12]:
detections_pred = full_inference_pipeline(model, test_loader, device)
print("Pred boxes:", len(detections_pred))

detections_gt = full_gt_pipeline(test_loader)
print("GT boxes:", len(detections_gt))

mean_iou = evaluate_object_iou(detections_pred, detections_gt)


Pred boxes: 2101
GT boxes: 2085
Mean Object IoU : 0.35905378782920444
Correct detections (IoU>0.5): 755
